<a href="https://colab.research.google.com/github/yoonwanggyu/Alpaco_Project/blob/main/%EA%B0%9D%EC%B2%B4%EC%9D%B8%EC%8B%9D_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8(06.03~06.20)/%EB%B2%BC_%EC%83%9D%EC%9C%A1%EC%9D%B4%EC%83%81_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 벼 생육이상 Inference

## 모델 로드

In [5]:
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "nvidia/segformer-b3-finetuned-cityscapes-1024-1024"
# we are going to do whole inference, so no resizing of the image
processor = SegformerImageProcessor(do_resize=False)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# 모델의 헤드 클래스 수를 변경
model.config.num_labels = 6  # fine-tuning 데이터셋의 클래스 수에 맞게 변경
model.decode_head.classifier = torch.nn.Conv2d(768, model.config.num_labels, kernel_size=1)  # 768은 B3의 내부 차원 수
model.load_state_dict(torch.load("/content/best_model.pth", map_location=device))


# model.to(device)

UnpicklingError: invalid load key, '\xe3'.

In [ ]:
model.to(device)
model.eval()

## 이미지 로드

In [ ]:
from PIL import Image

# load image + ground truth map
image_path = "C:/Users/USER/Desktop/512x288/resized_valid_image/NIA_AgricultureAD_paddy_RGB_bottom_Gyeongsangnamdo_2110151007_day_sunny_001894.jpg"
image = Image.open(image_path)

image

## 이미지 전처리

In [ ]:
from PIL import Image
import torchvision.transforms as T
import torch

# 전처리 파이프라인 설정
def preprocess_image(image_path, transforms):
    # 이미지 파일을 열고 RGB로 변환
    image = Image.open(image_path).convert('RGB')

    # 전달된 전처리 변환을 이미지에 적용
    if transforms:
        image = transforms(image)

    return image

# 이미지 경로
image = Image.open(image_path)

# 전처리 변환 (훈련과 동일)
inference_transforms = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 이미지 전처리
processed_image = preprocess_image(image_path, inference_transforms)

# 차원 조정 (batch_size, C, H, W)
processed_image = processed_image.unsqueeze(0)  # 이는 모델에 입력하기 위해 필요한 배치 차원을 추가합니다.

processed_image.shape
processed_image= processed_image.to(device)

In [ ]:
import torch

with torch.no_grad():
  outputs = model(processed_image)
  logits = outputs.logits

In [ ]:
import torch.nn.functional as F
upsampled_logits = F.interpolate(logits, size=(288 ,512), mode="bilinear", align_corners=False)
# 소프트맥스를 적용하여 픽셀별 클래스 확률을 계산
probabilities = F.softmax(upsampled_logits, dim=1)

# 각 픽셀에서 최대 확률과 해당 인덱스를 계산
max_probs, predicted_classes = torch.max(probabilities, dim=1)

# 임계값 설정 (예: 0.5)
# threshold = 0.3

# 임계값 미만인 픽셀을 배경으로 설정
# predicted_classes[max_probs < threshold] = 0


# Tensor를 Numpy 배열로 변환
mask_np = predicted_classes.cpu().numpy()

## 시각화

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_segmentation(image, mask, alpha=1):
    """
    이미지와 세그멘테이션 마스크를 시각화합니다.
    :param image: PIL 이미지 또는 NumPy 배열
    :param mask: 세그멘테이션 마스크 (numpy 배열)
    :param alpha: 마스크의 투명도
    """
    if not isinstance(image, np.ndarray):
        image = np.array(image)

    mask = mask.squeeze()
    # 12개 클래스에 대한 색상 매핑 정의
    colors = np.array([
        [0, 0, 0],        # 배경
        [128, 64, 128],   # 클래스 1 - common_road
        [244, 35, 232],   # 클래스 2 - common_tree
        [70, 70, 70],     # 클래스 3 - field_corps
        [102, 102, 156],  # 클래스 4 - field_furrow
        [190, 153, 153],  # 클래스 5 - field_levee
        [153, 153, 153],  # 클래스 6 - orchard_road
        [250, 170, 30],   # 클래스 7 - orchard_tree
        [220, 220, 0],    # 클래스 8 - paddy_after_driving
        [107, 142, 35],   # 클래스 9 - paddy_before_driving
        [152, 251, 152],  # 클래스 10 - paddy_edge
        [70, 130, 180],   # 클래스 11 - paddy_rice
        [220, 20, 60]     # 클래스 12 - paddy_water
    ], dtype=np.uint8)

    colored_mask = colors[mask]
    combined = image.astype(np.float32) * (1 - alpha) + colored_mask.astype(np.float32) * alpha
    combined = combined.clip(0, 255).astype(np.uint8)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(combined)
    plt.title('Segmentation Overlay')
    plt.axis('off')

    plt.show()

# 이미지와 예측된 마스크 데이터를 이 함수에 전달하여 시각화
# image는 PIL 이미지, predicted_classes는 위 코드에서 계산된 예측 클래스 numpy 배열


## Ground Truth

In [ ]:
import os
import json
from skimage.draw import polygon2mask
import matplotlib.pyplot as plt


id2label = {
            0: 'background',
            1: 'common_road',
            2: 'common_tree',
            3: 'field_corps',
            4: 'field_furrow',
            5: 'field_levee',
            6: 'orchard_road',
            7: 'orchard_tree',
            8: 'paddy_after_driving',
            9: 'paddy_before_driving',
            10: 'paddy_edge',
            11: 'paddy_rice',
            12: 'paddy_water'
        }

label2id = {v: k for k, v in id2label.items()}

image = Image.open(image_path).convert('RGB')
mask = np.full((image.height, image.width), 0, dtype=np.int32)  # 초기값을 0으로 설정
image_np=np.array(image)

base_path= "C:/Users/USER/Desktop/512x288/resized_valid_annotations"
annotation_path= os.path.join(base_path, image_path.split('/')[-1].split('.')[0]+'.json')

with open(annotation_path, 'r') as f:
    img_info= json.load(f)

for obj in img_info['objects']:
    class_id = label2id.get(obj['label'])
    for pos in obj['position']:
        coords = [(y, x) for x, y in zip(pos[::2], pos[1::2])] #pos[::2]는 짝수 인덱스(모든 x 좌표), pos[1::2]는 홀수 인덱스(모든 y 좌표)
        # print(f"Coords for {obj['label']} with class ID {class_id}: {coords}")  # 디버깅용 좌표 출력

            # 좌표가 이미지 경계를 벗어나는지 확인
        out_of_bounds_coords = [(x, y) for y, x in coords if x < 0 or x >= image.width or y < 0 or y >= image.height]
        if out_of_bounds_coords:
            #print(f"Warning: Some coordinates for {obj['label']} are out of image bounds: {out_of_bounds_coords}")
            coords = [(max(0, min(image.height - 0.1, y)), max(0, min(image.width - 0.1, x))) for y, x in coords] # 이미지 벗어나는 좌표 클리핑



        poly_mask = polygon2mask((image.height, image.width), coords)

        # 디버깅용으로 poly_mask가 True인 위치 출력
        # true_indices = np.where(poly_mask)
        # true_positions = list(zip(true_indices[0], true_indices[1]))
        # print(f"True positions in poly_mask: {true_positions[:10]}")  # 처음 10개의 위치만 출력

        mask[poly_mask] = class_id

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(image_np)
plt.title("Image")

plt.subplot(1, 2, 2)
plt.imshow(mask, cmap='gray')
plt.title("Mask")

plt.show()